In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.linear_model import Ridge

In [2]:
# Set parameters for linear regression

# ridge parameters for random search
fit_intercept = [True, False]
copy_X = [True, False]
alpha = [0.1, 0.5, 1, 2, 5, 10, 20, 50, 100]
tol = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']

# Create the random grid
random_grid = {'fit_intercept': fit_intercept,
                'copy_X': copy_X,
                'alpha': alpha,
                'tol': tol,
                'solver': solver}


In [3]:
#Define MASE Metric
def mean_absolute_scaled_error(y_true, y_pred):
    mase=0
    # Define numerator as the forecast error
    numerator = (np.abs(y_true - y_pred))

    # Define denominator as the mean absolute error of the in-sample one-step naive forecast
    y_true_ohne_1 = y_true[1:].reset_index(drop=True)
    y_true_ohne_ende = y_true[:-1].reset_index(drop=True)
    denominator = np.mean(np.abs(y_true_ohne_1 - y_true_ohne_ende))

    mase = np.mean(np.abs(numerator / denominator))

    return mase

scorer_mase= make_scorer(mean_absolute_scaled_error, greater_is_better=False)


In [4]:
# Load the data
data = pd.read_parquet('/Users/paddy/Documents/GitHub/Masterthesis_ML/data/03_30min_dataset.parquet')

# Convert the date column to datetime
data['date'] = pd.to_datetime(data['date']) #,format='%d/%m/%y %H:%M:%S').dt.strftime('%Y-%m-%d %H:%M:%S') 


In [5]:
#Feature engineering
# Create a new column for the time
data['time'] = [x for x in range(0, len(data))]

# Create the lagged features
lags = 48 + 1

# Create the lagged features
for i in range(1, lags):
    data['lag' + str(i)] = data['count'].shift(i)

# Drop the first three rows
data = data.dropna().reset_index(drop=True)

# Define the feature columns and the target column
feature_cols = [ f'lag{i}' for i in range(1, lags)]
target_col = 'y'

# Drop nan values
data = data.dropna()

# Rename column count to y
data = data.rename(columns={'count': 'y'})

In [6]:
# Set y to the last column
cols = list(data.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('y')) #Remove y from list
data = data[cols+['y']] #Create new dataframe with columns in the order you want

# drop the date column
train_data = np.delete(data, 0, 1) 

# Split the data into X and y
X_train, y_train = train_data[:, :-1], train_data[:, -1]


In [7]:
#vgl: https://lightrun.com/answers/scikit-learn-scikit-learn-grid_search-feeding-parameters-to-scorer-functions

# X and y to pandas dataframe
X_train = pd.DataFrame(X_train)
y_train = pd.Series(y_train)


In [8]:
# Set Cross Validation to 5 iterations
tscv = TimeSeriesSplit(n_splits=5)

In [9]:
# Instantiate model
model = Ridge()

search = RandomizedSearchCV(estimator = model, 
                               param_distributions = random_grid, 
                               n_iter = 25, 
                               cv = tscv,
                               refit=True, 
                               verbose=3, 
                               random_state=42, 
                               n_jobs = -1, 
                               scoring=scorer_mase, #make_scorer(scorer_mase, greater_is_better=True), #'neg_root_mean_squared_error', #
                               error_score=np.nan)


In [10]:
# Fit the random search model
search.fit(X_train, y_train)

Fitting 5 folds for each of 25 candidates, totalling 125 fits


[CV 1/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-0.789 total time=   0.0s
[CV 2/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-0.848 total time=   0.0s
[CV 3/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-0.817 total time=   0.0s
[CV 4/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-0.789 total time=   0.0s
[CV 4/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-0.789 total time=   0.1s
[CV 5/5] END alpha=10, copy_X=False, fit_intercept=True, solver=sparse_cg, tol=0.001;, score=-0.770 total time=   0.0s
[CV 1/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-0.790 total time=   0.0s
[CV 5/5] END alpha=0.5, copy_X=True, fit_intercept=True, solver=cholesky, tol=0.01;, score=-0.770 total time=   0.0s
[CV 2/5] END alpha=0.5, copy_X=True, fit_intercept=Tru

RandomizedSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                   estimator=Ridge(), n_iter=25, n_jobs=-1,
                   param_distributions={'alpha': [0.1, 0.5, 1, 2, 5, 10, 20, 50,
                                                  100],
                                        'copy_X': [True, False],
                                        'fit_intercept': [True, False],
                                        'solver': ['auto', 'svd', 'cholesky',
                                                   'lsqr', 'sparse_cg', 'sag',
                                                   'saga'],
                                        'tol': [0.0001, 0.001, 0.01, 0.1, 1, 10,
                                                100]},
                   random_state=42,
                   scoring=make_scorer(mean_absolute_scaled_error, greater_is_better=False),
                   verbose=3)

In [11]:
print('Best Score: %s' % search.best_score_)
print('Best Hyperparameters: %s' % search.best_params_)
print('Best Model: %s' % search.best_estimator_)
print('Best Index: %s' % search.best_index_)
print('CV Results: %s' % search.cv_results_)
print('Refit Time: %s' % search.refit_time_)
print('Scorer: %s' % search.scorer_)

Best Score: -0.8027533326416207
Best Hyperparameters: {'tol': 0.001, 'solver': 'sparse_cg', 'fit_intercept': True, 'copy_X': False, 'alpha': 10}
Best Model: Ridge(alpha=10, copy_X=False, solver='sparse_cg', tol=0.001)
Best Index: 0
CV Results: {'mean_fit_time': array([0.0248322 , 0.02359118, 0.02833853, 0.01738276, 0.01345301,
       0.01230283, 1.69453883, 0.01731658, 0.01696501, 0.03439512,
       0.01414356, 0.01776657, 0.01409931, 0.02047524, 0.02042904,
       1.24691095, 0.0158155 , 0.03828526, 0.02168384, 0.01857605,
       0.01540279, 0.01458364, 0.01881762, 0.03176656, 0.0131156 ]), 'std_fit_time': array([0.01502874, 0.00994117, 0.01144636, 0.00704271, 0.00476681,
       0.00450656, 1.38945856, 0.00879521, 0.00644429, 0.01699411,
       0.00688164, 0.0067397 , 0.00570578, 0.01208006, 0.00846063,
       0.83405512, 0.00683872, 0.01738689, 0.00843282, 0.00803891,
       0.00551062, 0.00415824, 0.00777648, 0.01378564, 0.00459075]), 'mean_score_time': array([0.00774856, 0.00707445